In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import gradio as gr

from show_homes import PROJECT_DIR, config
from show_homes.pipeline import show_homes_network

TypeError: 'module' object is not subscriptable

In [ ]:
config

<module 'show_homes.config' from '/Users/juliasuter/Documents/repositories/show_homes/show_homes/config/__init__.py'>

In [19]:
from show_homes import config, _base_config_path, get_yaml_config

config = get_yaml_config(_base_config_path)
config


{'LOCAL_DATA_DIR': '/Users/juliasuter/Documents/ASF_data/',
 'FIG_OUT_PATH': '/outputs/figures/',
 'DIST_DUR_OUT_DATA_PATH': '/outputs/data/dist_and_durations/',
 'DIST_MATRIX_OUT_DATA_PATH': '/outputs/data/dist_matrices/',
 'HOST_VIS_CON_OUT_DATA_PATH': '/outputs/data/host_vis_connections/',
 'SHOW_HOME_DATA_OUT_DATA_PATH': '/outputs/data/show_home_data/',
 'KEPLER_CONFIG_PATH': '/outputs/maps/config/',
 'MAPS_OUT_PATH': '/outputs/maps/',
 'GRADIO_OUT_MAPS_PATH': '/show_homes/analysis/maps/'}

In [17]:
df = pd.read_csv(config.SHOW_HOME_DATA_OUT_DATA_PATH / epc_for_show_homes.csv')

In [20]:
show_homes_network.compute_network_measure(df, 'Detached House', True, 1,5, 5, 6, 30, 'GB', verbose=True)

GB
0.05
Detached House
0.01
30
30
Detached House
True
After subsampling:
# Props without HPs: 1391588
# Props with HPs: 39147
Before subsampling:
# Props without HPs: 69579
# Props with HPs: 391


OSError: Cannot save file into a non-existent directory: '/Users/juliasuter/Documents/repositories/show_homes/outputs/data/connections'

In [ ]:

def G(property_type, same_prop_type, 
                            host_ratio, visitor_ratio, 
                            v_max, n_open_days, d_max, local_auth, verbose=True):
    
    return show_homes_network.compute_network_measure(df, property_type, same_prop_type, 
                            host_ratio, visitor_ratio, 
                            v_max, n_open_days, d_max, local_auth, verbose=True)
 
    
property_types = ['Flat', 'Semi-detached house', 'Detached House', 'Terraced House', 'Any']
local_authorities = sorted(list(df['LOCAL_AUTHORITY_LABEL'].unique())) + ['GB']
    
demo = gr.Interface(
    fn=G,
    inputs=[gr.inputs.Radio(property_types, label='Property Type', default='Detached House'), 
            gr.inputs.Radio([True, False], label='Show home of same property', default=True), 
            gr.inputs.Slider(0, 100, default=1, step=1, label='Host ratio (%)'), 
            gr.inputs.Slider(0, 100, default=5, step=1, label='Visitor ratio (%)'),
            gr.inputs.Slider(1, 50, default=5, step=1, label='Max visitors'), 
            gr.inputs.Slider(1, 50, default=6, step=1, label='Number of slots/open days'), 
            gr.inputs.Slider(1, 50, default=35, step=1, label='Max distance'),
            gr.inputs.Dropdown(local_authorities, default='Manchester', label='Local authorities')],
    outputs=["text", "html"],
    title='Network of Show Homes',)

demo.launch(share=True)